### Resolución con PuLP

In [1]:
from pulp import *
import pandas as pd

In [2]:
projects = ["P1", "P2", "P3", "P4", "P5"]
years = ["Y1", "Y2", "Y3"]
proficts = pd.Series(index = projects, data=[20, 40, 20, 15, 30])
years_expenses = pd.DataFrame(index = projects, columns=years, data=[[5,1,8], [4,7,10], [3,9,2], [7,4,1], [8,6,10]])
budgets = pd.Series(index = years, data=[25] * 3)
years_expenses

,Y1,Y2,Y3
P1,5,1,8
P2,4,7,10
P3,3,9,2
P4,7,4,1
P5,8,6,10


In [19]:
# Crear problema de maximización
prob = LpProblem("Projects", LpMaximize)
prob
# Crear variables de asientos de cada tarifa 
select = LpVariable.dicts("Project_Selected", indices=projects, lowBound=0, cat=LpBinary) 
select

{'P1': Project_Selected_P1,
 'P2': Project_Selected_P2,
 'P3': Project_Selected_P3,
 'P4': Project_Selected_P4,
 'P5': Project_Selected_P5}

In [20]:
# Añadir función objetivo al problema
prob += lpSum([proficts[i] * select[i] for i in projects])
prob

Projects:
MAXIMIZE
20*Project_Selected_P1 + 40*Project_Selected_P2 + 20*Project_Selected_P3 + 15*Project_Selected_P4 + 30*Project_Selected_P5 + 0
VARIABLES
0 <= Project_Selected_P1 <= 1 Integer
0 <= Project_Selected_P2 <= 1 Integer
0 <= Project_Selected_P3 <= 1 Integer
0 <= Project_Selected_P4 <= 1 Integer
0 <= Project_Selected_P5 <= 1 Integer

In [21]:
# Añadir restricciones de no sobrepasar topes de demandas
for y in years:
    prob += (lpSum([years_expenses.at[p, y] * select[p] for p in projects]) <= budgets.at[y])

Projects:
MAXIMIZE
20*Project_Selected_P1 + 40*Project_Selected_P2 + 20*Project_Selected_P3 + 15*Project_Selected_P4 + 30*Project_Selected_P5 + 0
SUBJECT TO
_C1: 5 Project_Selected_P1 + 4 Project_Selected_P2 + 3 Project_Selected_P3
 + 7 Project_Selected_P4 + 8 Project_Selected_P5 <= 25

_C2: Project_Selected_P1 + 7 Project_Selected_P2 + 9 Project_Selected_P3
 + 4 Project_Selected_P4 + 6 Project_Selected_P5 <= 25

_C3: 8 Project_Selected_P1 + 10 Project_Selected_P2 + 2 Project_Selected_P3
 + Project_Selected_P4 + 10 Project_Selected_P5 <= 25

VARIABLES
0 <= Project_Selected_P1 <= 1 Integer
0 <= Project_Selected_P2 <= 1 Integer
0 <= Project_Selected_P3 <= 1 Integer
0 <= Project_Selected_P4 <= 1 Integer
0 <= Project_Selected_P5 <= 1 Integer

In [22]:
# Comprobar que el problema está bien formulado
prob

Projects:
MAXIMIZE
20*Project_Selected_P1 + 40*Project_Selected_P2 + 20*Project_Selected_P3 + 15*Project_Selected_P4 + 30*Project_Selected_P5 + 0
SUBJECT TO
_C1: 5 Project_Selected_P1 + 4 Project_Selected_P2 + 3 Project_Selected_P3
 + 7 Project_Selected_P4 + 8 Project_Selected_P5 <= 25

_C2: Project_Selected_P1 + 7 Project_Selected_P2 + 9 Project_Selected_P3
 + 4 Project_Selected_P4 + 6 Project_Selected_P5 <= 25

_C3: 8 Project_Selected_P1 + 10 Project_Selected_P2 + 2 Project_Selected_P3
 + Project_Selected_P4 + 10 Project_Selected_P5 <= 25

VARIABLES
0 <= Project_Selected_P1 <= 1 Integer
0 <= Project_Selected_P2 <= 1 Integer
0 <= Project_Selected_P3 <= 1 Integer
0 <= Project_Selected_P4 <= 1 Integer
0 <= Project_Selected_P5 <= 1 Integer

In [25]:
# Resolver el problema, pintar la solución y ver cuánto da de objetivo
status = prob.solve()
print("Status: ", LpStatus[status])
for v in prob.variables():
  val = value(v)
  print(v.name, "=", v.varValue)
round(value(prob.objective), 2)

Status:  Optimal
Project_Selected_P1 = 1.0
Project_Selected_P2 = 1.0
Project_Selected_P3 = 1.0
Project_Selected_P4 = 1.0
Project_Selected_P5 = 0.0


95.0